# ETL MOVIES 

In [1]:
#Primero importaremos todas las herramientas que utilizaremos en el proceso de ETL

import ast 
import pandas as pd
import numpy as np


In [2]:
#Cargamos los datasets en variables con el metodo de pandas pd.read_csv()
movies_df = pd.read_csv('/Users/pabloclementi/Desktop/Proyecto1/Proyecto_final_1/data_original/movies_dataset.csv') 

#Recuerda , debes bajar los datasets originales desde el enlace proporcionado en el README


/var/folders/4k/b142n1x53kq0891h836m2hh80000gn/T/ipykernel_7105/4174696762.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('/Users/pabloclementi/Desktop/Proyecto1/Proyecto_1/data/movies_dataset.csv')


In [8]:
def expandircolumnas(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        # Expandir la columna en filas
        df = df.explode(column)
        # Normalizar la columna expandida
        col_df = pd.json_normalize(df[column])
        # Renombrar las columnas normalizadas para evitar superposiciones
        col_df = col_df.add_prefix(f'{column}')
        # Concatenar las columnas normalizadas con el DataFrame original
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['genres', 'production_companies', 'production_countries', 'spoken_languages','belongs_to_collection']

In [9]:
# Desanidar y expandir las columnas en movies_df
movies_df = expandircolumnas(movies_df, columns_to_expand)


In [10]:
# Verificar las primeras filas del DataFrame expandido
movies_df.head(1)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English


In [13]:
#Realizamos una funcion para eliminar espacios en blanco en las columnas
def trim_columnas(df):
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    return df

In [14]:
trim_columnas(movies_df)
# trim_columnas(credits_sample)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
2,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
3,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
4,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582881,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,3.8,6.0,28.0,Action,American World Pictures,6165.0,US,United States of America,en,English
582882,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,3.8,6.0,18.0,Drama,American World Pictures,6165.0,US,United States of America,en,English
582883,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,3.8,6.0,53.0,Thriller,American World Pictures,6165.0,US,United States of America,en,English
582884,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,0.0,0.0,NaN,NaN,Yermoliev,88753.0,RU,Russia,NaN,NaN


In [15]:
# Contar los valores nulos en las columnas 'revenue' y 'budget'
null_counts = movies_df[['revenue', 'budget']].isnull().sum()

print("Valores nulos en 'revenue' y 'budget':")
print(null_counts)

Valores nulos en 'revenue' y 'budget':
revenue    27
budget      0
dtype: int64


In [16]:
#Los valores nulos de los campos `revenue`,`budget`deben ser rellenados por el número `0`
movies_df['revenue'] = movies_df['revenue'].fillna(0)
movies_df['budget'] = movies_df['budget'].fillna(0)
#Luego de ejecutar estos codigos, verificamos ejecutando nuevamente el codigo anterior. 

In [17]:
#Eliminar filas con valores nulos en 'release_date'
movies_df.dropna(subset=['release_date'], inplace=True)

In [18]:
#Convertir 'release_date' a formato AAAA-mm-dd y crear 'release_year'
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')
movies_df['release_year'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.year

In [19]:
#Eliminar filas donde 'release_date' es nulo o no pudo convertirse
movies_df = movies_df.dropna(subset=['release_date'])



In [20]:
# Reiniciar los índices después de la eliminación
movies_df.reset_index(drop=True, inplace=True)

In [21]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'],format='%Y-%m-%d')

/var/folders/4k/b142n1x53kq0891h836m2hh80000gn/T/ipykernel_6273/3631002381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['release_date'] = pd.to_datetime(movies_df['release_date'],format='%Y-%m-%d')


In [22]:
movies_df['release_date']


0        1995-10-30
1        1995-10-30
2        1995-10-30
3        1995-10-30
4        1995-10-30
            ...    
582663   2003-08-01
582664   2003-08-01
582665   2003-08-01
582666   1917-10-21
582667   2017-06-09
Name: release_date, Length: 582668, dtype: datetime64[ns]

In [23]:
movies_df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'genresid', 'genresname',
       'production_companiesname', 'production_companiesid',
       'production_countriesiso_3166_1', 'production_countriesname',
       'spoken_languagesiso_639_1', 'spoken_languagesname', 'release_year'],
      dtype='object')

In [24]:
# Mostrar el tipo de dato de las columnas 'budget' y 'revenue'
print("Tipo de dato de 'budget':", movies_df['budget'].dtype)
print("Tipo de dato de 'revenue':", movies_df['revenue'].dtype)

Tipo de dato de 'budget': object
Tipo de dato de 'revenue': float64


In [25]:

# Limpiar valores no numéricos en la columna 'budget' y reemplazar con 0
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce').fillna(0)

# Convertir la columna 'budget' a float64
movies_df['budget'] = movies_df['budget'].astype('float64')

# Mostrar el DataFrame después de la limpieza
movies_df.head()

/var/folders/4k/b142n1x53kq0891h836m2hh80000gn/T/ipykernel_6273/4174079110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce').fillna(0)
/var/folders/4k/b142n1x53kq0891h836m2hh80000gn/T/ipykernel_6273/4174079110.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['budget'] = movies_df['budget'].astype('float64')


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname,release_year
0,False,30000000.0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0
1,False,30000000.0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0
2,False,30000000.0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0
3,False,30000000.0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0
4,False,30000000.0,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,5415.0,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0


In [26]:
#Crear la columna con el retorno de inversión, llamada `return` con los campos `revenue` y `budget`, dividiendo estas dos últimas `revenue / budget`, cuando no hay datos disponibles para calcularlo, deberá tomar el valor `0`
movies_df['return'] = movies_df['revenue'] / movies_df['budget']
# Reemplazar valores infinitos por 0
movies_df['return'].replace([np.inf, -np.inf], 0, inplace=True)
    
movies_df['return'].isnull().sum() #Contamos cuantos datos nullos hay por no poder calcularse 
#Tranasformamos los datos nullos a 0 
movies_df['return'] = movies_df['return'].fillna(0)
movies_df['return'].isnull().sum() #Verificamos que ya no hay datos nulos. 
#movies_sample.head(3) #Verificamos la creacion de la columna `return`.
#movies_sample.info() 

/var/folders/4k/b142n1x53kq0891h836m2hh80000gn/T/ipykernel_6273/3123376648.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['return'] = movies_df['revenue'] / movies_df['budget']
/var/folders/4k/b142n1x53kq0891h836m2hh80000gn/T/ipykernel_6273/3123376648.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inpla

0

In [27]:
# Vemos todas las columnas del data frame 
movies_df.columns


Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'genresid', 'genresname',
       'production_companiesname', 'production_companiesid',
       'production_countriesiso_3166_1', 'production_countriesname',
       'spoken_languagesiso_639_1', 'spoken_languagesname', 'release_year',
       'return'],
      dtype='object')

In [28]:
#Eliminar las columnas que no serán utilizadas, `video`,`imdb_id`,`adult`,`original_title`,`poster_path` y `homepage`. 
movies_df = movies_df.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'])




In [29]:
movies_df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname,release_year,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
1,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
2,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
3,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
4,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801


In [31]:
# Verificar cuántas filas duplicadas hay en el DataFrame, y pasar a eliminarlas del data frame
filas_duplicadas = movies_df.duplicated().sum()
print("Número de filas duplicadas:", filas_duplicadas) 
movies_df = movies_df.drop_duplicates() 
#Verificar nuevamente los duplicados. 
filas_duplicadas=movies_df.duplicated().sum()
print("Luego de eliminar los duplicados , el numero de filas duplicadas es ,",filas_duplicadas)

Número de filas duplicadas: 163710
Luego de eliminar los duplicados , el numero de filas duplicadas es , 0


In [32]:
# Convertir a cadenas y eliminar valores NaN
movies_df['spoken_languagesname'] = movies_df['spoken_languagesname'].astype(str)
movies_df['genresname'] = movies_df['genresname'].astype(str)

In [33]:
movies_df

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname,release_year,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
4,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
8,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10751.0,Family,Pixar Animation Studios,3.0,US,United States of America,en,English,1995.0,12.451801
12,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,12.0,Adventure,TriStar Pictures,559.0,US,United States of America,en,English,1995.0,4.043035
13,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,12.0,Adventure,TriStar Pictures,559.0,US,United States of America,fr,Français,1995.0,4.043035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582663,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,28.0,Action,American World Pictures,6165.0,US,United States of America,en,English,2003.0,0.000000
582664,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,18.0,Drama,American World Pictures,6165.0,US,United States of America,en,English,2003.0,0.000000
582665,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,53.0,Thriller,American World Pictures,6165.0,US,United States of America,en,English,2003.0,0.000000
582666,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,...,NaN,nan,Yermoliev,88753.0,RU,Russia,NaN,nan,1917.0,0.000000


In [34]:
# Agrupar por 'id' y agregar los idiomas y géneros
grouped = movies_df.groupby('id').agg({
    'spoken_languagesname': lambda x: ', '.join(set(x)),
    'genresname': lambda x: ', '.join(set(x))
}).reset_index()

# Seleccionar otras columnas que deseas mantener (ejemplo: budget, revenue, etc.)
other_columns = ['title','budget', 'revenue', 'overview','release_date', 'vote_average', 'vote_count','popularity','runtime', 'status', 'tagline', 'production_companiesname', 'production_countriesname', 'release_year', 'return']

# Seleccionar las columnas únicas que no necesitas agrupar
single_value_columns = movies_df.groupby('id')[other_columns].first().reset_index()

# Combinar idiomas, generos y otras columnas en un solo DataFrame por 'id'
merged_movies = pd.merge(grouped, single_value_columns, on='id', how='left')

# Mostrar el resultado
#merged_movies.head(15) 
merged_movies.shape #Aqui notamos una amplia reduccion ... 

(45346, 18)

In [35]:
merged_movies.isnull().sum()#Verifica la cantidad de datos nulos en el Df. 

id                              0
spoken_languagesname            0
genresname                      0
title                           0
budget                          0
revenue                         0
overview                      941
release_date                    0
vote_average                    0
vote_count                      0
popularity                      0
runtime                       246
status                         80
tagline                     24959
production_companiesname    11789
production_countriesname     6208
release_year                    0
return                          0
dtype: int64

In [36]:
#Completamos los datos faltantes de las columnas con datos null. 
merged_movies['title'] = merged_movies['title'].fillna('SinDato')
merged_movies['overview'] = merged_movies['overview'].fillna('SinDato')
merged_movies['release_date'] = merged_movies['release_date'].fillna('00/00/0000')
merged_movies['vote_average'] = merged_movies['vote_average'].fillna('SinDato')
merged_movies['vote_count'] = merged_movies['vote_count'].fillna('SinDato')
merged_movies['runtime'] = merged_movies['runtime'].fillna('SinDato')
merged_movies['status'] = merged_movies['status'].fillna('SinDato')
merged_movies['tagline'] = merged_movies['tagline'].fillna('SinDato')
merged_movies['production_companiesname'] = merged_movies['production_companiesname'].fillna('SinDato')
merged_movies['production_countriesname'] = merged_movies['production_countriesname'].fillna('SinDato')
merged_movies['release_year'] = merged_movies['release_year'].fillna(0000)# Al no haber datos de fecha ponemos año 0000

In [37]:
# # El DF merged_movies lo guardaremos como archivo con nombre Movies_Limpio.
# merged_movies.to_csv('Movies_Limpio.csv', index=False)
# merged_movies.info() #Memory usage 5.9 MB
# # merged_movies.columns
#merged_movies.describe()
#merged_movies.isnull().sum()
merged_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        45346 non-null  object        
 1   spoken_languagesname      45346 non-null  object        
 2   genresname                45346 non-null  object        
 3   title                     45346 non-null  object        
 4   budget                    45346 non-null  float64       
 5   revenue                   45346 non-null  float64       
 6   overview                  45346 non-null  object        
 7   release_date              45346 non-null  datetime64[ns]
 8   vote_average              45346 non-null  float64       
 9   vote_count                45346 non-null  float64       
 10  popularity                45346 non-null  object        
 11  runtime                   45346 non-null  object        
 12  status            

In [38]:
# Se detectaron valores no numericos en ID> Encuentra el índice donde se encuentra el valor no numérico en 'id'
index_error = merged_movies[merged_movies['id'].apply(lambda x: not str(x).isdigit())].index

# Muestra el índice y el valor problemático
print("Índice del valor problemático:", index_error)
print("Valor problemático:", merged_movies.loc[index_error, 'id'])

Índice del valor problemático: Index([], dtype='int64')
Valor problemático: Series([], Name: id, dtype: object)


In [39]:
# Eliminar la fila con el índice del valor problemático
merged_movies.drop(index_error, inplace=True)

In [40]:
#convertimos lo datos de la columna id a int 
merged_movies['id'] = merged_movies['id'].astype('int64')

#Pasar la columna release date a formato datatime 
# # merged_movies['release_date'] = pd.to_datetime(merged_movies['release_date'])
# merged_movies['release_date'] = pd.to_datetime(merged_movies['release_date'], errors='coerce')
#Debo volver a object la columna de release date 
#merged_movies['release_date'] = merged_movies['release_date'].astype(str)
#Convertir las columnas vote_average y vote_count a float   
merged_movies['vote_average'] = merged_movies['vote_average'].astype('float64')
merged_movies['vote_count'] = merged_movies['vote_count'].astype('float64')
merged_movies['popularity'] = merged_movies['popularity'].astype('float64')

In [41]:
#Sobreescribimos los cambios en el archivo Movies_limpio.csv
merged_movies.to_csv('Movies_Limpio.csv', index=False) 
merged_movies.info() 
#Al tratar de ejecutar la api con la funcion para ver la cantidad de peliculas estrenadas por mes me da error entonces devemos seguir trabajando sobre la columna 'release_date'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        45346 non-null  int64         
 1   spoken_languagesname      45346 non-null  object        
 2   genresname                45346 non-null  object        
 3   title                     45346 non-null  object        
 4   budget                    45346 non-null  float64       
 5   revenue                   45346 non-null  float64       
 6   overview                  45346 non-null  object        
 7   release_date              45346 non-null  datetime64[ns]
 8   vote_average              45346 non-null  float64       
 9   vote_count                45346 non-null  float64       
 10  popularity                45346 non-null  float64       
 11  runtime                   45346 non-null  object        
 12  status            